## Sentiment Analysis of Rotten Tomatoes Reviews using Naive Bayes

#### Some imports to make code compatible with Python 2 as well as 3

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#### Import nltk, the Natural Language Processing Toolkit

This is one of the most popular packages for natural language processing on text data. It has APIs to access a large corpus of documents and other lexical resources

In [4]:
import numpy as np
import nltk

In [5]:
print(np.__version__)
print(nltk.__version__)

1.13.1
3.2.1


### Extract the review text and corresponding sentiment label from the review files

The dataset is available for download from this site: http://www.cs.cornell.edu/people/pabo/movie-review-data/

Search for "sentence polarity dataset v1.0 (includes sentence polarity dataset README v1.0: 5331 positive and 5331 negative processed sentences / snippets. Introduced in Pang/Lee ACL 2005. Released July 2005."

This is the dataset to download, unzip and untar.

**Store the files with positive and negative reviews (rt-polarity.pos and rt-polarity.neg) in the same directory as this code**


In [6]:
def get_reviews(path, positive=True):
    label = 1 if positive else 0

    reviews = []

    with open(path, 'r') as f:
        review_text = f.readlines()
        
    for text in review_text:
        # Return a tuple of the review text and a label for whether it 
        # is a positive or negative review
        reviews.append((text, label))

    return reviews           

def extract_reviews():
    positive_reviews = get_reviews("rt-polarity.pos", positive=True)
    negative_reviews = get_reviews("rt-polarity.neg", positive=False)

    return positive_reviews, negative_reviews

In [7]:
positive_reviews, negative_reviews = extract_reviews()

In [8]:
positive_reviews[:2]

[('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . \n',
  1),
 ('the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth . \n',
  1)]

In [9]:
TRAIN_DATA = 5000
TOTAL_DATA = 6000

train_reviews = positive_reviews[:TRAIN_DATA] + negative_reviews[:TRAIN_DATA]

test_positive_reviews = positive_reviews[TRAIN_DATA:TOTAL_DATA]
test_negative_reviews = negative_reviews[TRAIN_DATA:TOTAL_DATA]

#### Get a list of all the unque words in the dataset, the vocabulary

In [10]:
def get_vocabulary(train_reviews):
    words_set = set()
    
    for review in train_reviews:
        words_set.update(review[0].split())
    
    return list(words_set)

vocabulary = get_vocabulary(train_reviews)

In [11]:
len(vocabulary)

20704

In [12]:
vocabulary[:5]

['unimaginative', 'writings', 'bad-boy', 'ryoko', 'foul']

### Represent the words in the review as a feature vector

* *review_text* The review in text form

Each review is represented as a dictionary where keys are all words in the vocabulary. The values associated with each key is True if the word is present in the review.

In [17]:
def extract_features(review_text):
    # Split the review into words, and create a set of the words
    review_words = set(review_text.split())
    
    features = {}
    for word in vocabulary:
        features[word] = (word in review_words)
        
    return features    

#### Map feature vector to labels

* *extract_features* Function to extract the features in feature vector form
* *train_reviews* Training dataset, a list of tuples of the form (review_text, label)

In [15]:
train_features = nltk.classify.apply_features(extract_features, train_reviews)

#### Train the classifier on the training data

In [16]:
trained_classifier = nltk.NaiveBayesClassifier.train(train_features)

In [18]:
def sentiment_calculator(review_text):
    features = extract_features(review_text)
    return trained_classifier.classify(features)

In [19]:
sentiment_calculator("What an amazing movie!")

1

In [20]:
sentiment_calculator("What a terrible movie")

0

#### Classify and measure the accuracy of the model on test data

In [21]:
def classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator):
    positive_results = [sentiment_calculator(review[0]) for review in test_positive_reviews]
    negative_results = [sentiment_calculator(review[0]) for review in test_negative_reviews]
    
    true_positives = sum(x > 0 for x in positive_results)
    true_negatives = sum(x == 0 for x in negative_results)
    
    percent_true_positive = float(true_positives) / len(positive_results)
    percent_true_negative = float(true_negatives) / len(negative_results)

    total_accurate = true_positives + true_negatives
    total = len(positive_results) + len(negative_results)

    print("Accuracy on positive reviews = " +"%.2f" % (percent_true_positive * 100) + "%")
    print("Accurance on negative reviews = " +"%.2f" % (percent_true_negative * 100) + "%")
    print("Overall accuracy = " + "%.2f" % (total_accurate * 100/ total) + "%")

In [22]:
classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator)

Accuracy on positive reviews = 78.25%
Accurance on negative reviews = 80.66%
Overall accuracy = 79.46%
